In [5]:
import requests

import altair as alt
import pandas as pd

Download treatment duration data
Sample data is in `behandelduur-distribution-20200425.json



In [6]:
treatment_duration = requests.get("https://www.stichting-nice.nl/covid-19/public/behandelduur-distribution/").json()

In [90]:
# treatment_duration:
#
# [0]: levend_nog_opgenomen_ziekenhuis
# [1]: nog_opgenomen_ic
# [2]: levend_ziekenhuis_verlaten
# [3]: overleden
levend_nog_opgenomen_ziekenhuis = treatment_duration[0]
nog_opgenomen_ic = treatment_duration[1]
levend_ziekenhuis_verlaten = treatment_duration[2]
overleden = treatment_duration[3]

def parse_tuples(tpls, name: str) -> pd.DataFrame:
    # Push [0,0] to front of data so it starts at 0
    tpls[-1] = ((0,0))
    return pd.DataFrame(tpls, columns=['days', name]).set_index('days')

treatment_duration_df = parse_tuples(levend_nog_opgenomen_ziekenhuis, 'levend_nog_opgenomen_ziekenhuis')\
    .join(parse_tuples(nog_opgenomen_ic, 'nog_opgenomen_ic'), how='outer')\
    .join(parse_tuples(levend_ziekenhuis_verlaten, 'levend_ziekenhuis_verlaten'), how='outer')\
    .join(parse_tuples(overleden, 'overleden'), how='outer')\
    .fillna(0)
    # Fill all the 0's after joining because it may lengthen the data


In [91]:
# Cumulative sum by column
treatment_duration_df.cumsum().head()

,levend_nog_opgenomen_ziekenhuis,nog_opgenomen_ic,levend_ziekenhuis_verlaten,overleden
days,,,,
0,0,0,0.0,0.0
1,20,7,3.0,24.0
2,40,22,6.0,54.0
3,59,31,20.0,80.0
4,72,42,31.0,116.0


In [108]:
# Take cumulative sum
# move to long format so we can map a column to color channel later
chart_data_bar = treatment_duration_df.reset_index().melt(id_vars='days')

alt.Chart(chart_data_bar).mark_bar().encode(
    x=alt.X('days:O', axis=alt.Axis(title='Dagen na opname')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Aantal IC opnamen')),
    color=alt.Color('variable:N', legend=alt.Legend(title='Uitkomst'))
)

alt.Chart(...)

In [107]:
# Take cumulative sum
# move to long format so we can map a column to color channel later
chart_data = treatment_duration_df.cumsum().reset_index().melt(id_vars='days')

alt.Chart(chart_data).mark_line().encode(
    x=alt.X('days:O', axis=alt.Axis(title='Dagen na opname')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Aantal IC opnamen')),
    color=alt.Color('variable:N', legend=alt.Legend(title='Uitkomst'))
)

alt.Chart(...)

In [106]:
alt.Chart(chart_data).mark_area().encode(
    x=alt.X('days:O', axis=alt.Axis(title='Dagen na opname')),
    y=alt.Y('value:Q', axis=alt.Axis(title='Aantal IC opnamen')),
    color=alt.Color('variable:N', legend=alt.Legend(title='Uitkomst'))
)

alt.Chart(...)